In [1]:
import pandas as pd
import numpy as np

from gensim.models import Word2Vec
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# импортируем данные
train = pd.read_excel('semifinal_data/train.xlsx')
test = pd.read_excel('semifinal_data/test.xlsx')
descr = pd.read_excel('semifinal_data/description.xlsx')

In [3]:
# собираем всю словесную информацию вместе
train['Full_descr'] = train['Shrt_Desc'].values + ' ' +\
    train['GmWt_Desc1'].fillna('').values + ' ' +\
    train['GmWt_Desc2'].fillna('').values

train = train.drop(['Shrt_Desc', 'GmWt_Desc1', 'GmWt_Desc2'], axis = 1)

test['Full_descr'] = test['Shrt_Desc'].values + ' ' +\
    test['GmWt_Desc1'].fillna('').values + ' ' +\
    test['GmWt_Desc2'].fillna('').values

test = test.drop(['Shrt_Desc', 'GmWt_Desc1', 'GmWt_Desc2'], axis = 1)

In [4]:
def prepare_description(description):
    word_form = description.lower().split()
        
    res = []
    for i in word_form:
        res += i.split(',')
        
    for i in range(len(res)):
        try_to_find = res[i].find('w/')
        if (try_to_find != -1):
            res[i] = res[i][:try_to_find] + res[i][try_to_find + 2:]
        
    return res

# уберём подстроки 'w/' и приведем к нижнему регистру
word_information = train['Full_descr'].apply(prepare_description)

In [5]:
# возьмём контекст слов из книги с рецептами
text = []
for i in range(1, 14):
    with open(".\semifinal_files\\book_{}.txt".format(i), 'r') as file:
        text += [eval(file.read())]

# в датасете есть неудобные однобуквенные слова,
# от которых нужно избавиться
for i in range(len(text)):
    j = 0
    while j < len(text[i]):
        text[i][j] = text[i][j].lower()
        
        if len(text[i][j]) == 1:
            text[i].pop(j)
            j -= 1
        j += 1

In [6]:
# word2vec для слов из датасета
cook_book = Word2Vec(text, min_count=1, size=40, workers=5,
                 window=8, sg = 5, iter = 10)

In [7]:
# список всех слов
all_words = set()
for item in cook_book.wv.vocab:
    all_words.add(item)

Каждому типу продукта сопоставим слова из доступного словаря:\
1) Хлебобулочные изделия : 'bakery', 'bread', 'loaf', 'bun', 'sugar', 'cook', 'bake'\
2) Жидкость : 'liquid', 'water', 'juice', 'wine', 'cocktail'\
3) Молочная продукция : 'milk', 'dairy', 'yogurt', 'buttermilk'\
4) Мясная продукция : 'meat', 'lamb', 'pork', 'mutton', 'bone', 'raw'\
5) Овощи / фрукты : 'apple', 'orange', 'cucumber', 'tomato'

In [8]:
# функция, по которой определяется принадлежность к классу
def get_voting(description, key_words, min_confidence = 0.8):
    votes = 0
    sum_confidence = 0
    for word in description:
        if word in all_words:
            for key_word in key_words:
                sim = cook_book.wv.similarity(word, key_word)
                if sim >= min_confidence: 
                    sum_confidence += sim
                    votes += 1
    if not votes:
        return 0
    return sum_confidence / votes

In [9]:
# выделим нужные слова в label
label = [[]] * 6
label[1] = ['bakery', 'bread', 'loaf', 'bun', 'cook']
label[2] = ['liquid', 'water', 'juice']
label[3] = ['milk', 'dairy', 'curd', 'cream']
label[4] = ['meat', 'pork', 'fish', 'bone', 'chicken']
label[5] = ['apple', 'orange', 'cucumber', 'tomato']

train_votes = pd.DataFrame()
for i in range(1, 6):
    train_votes['{}'.format(i)] = word_information.apply(lambda x : get_voting(x, label[i]))
    
label_columns = ['{}'.format(i) for i in range(1, 6)]

# найденные классы для исходного train
train_res = train_votes[label_columns].apply(lambda x : x.argmax(), axis=1)

c:\users\shepe\python\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  app.launch_new_instance()


In [10]:
# посмоторим на распределение классов
np.unique(train_res.values, return_counts=True)

(array(['1', '2', '3', '4', '5'], dtype=object),
 array([2785,  932, 2033, 1512,  494], dtype=int64))

In [11]:
# посмотрим на случайные ответы
def foo(x):
    if x == '1':
        return "Хлебобулочные изделия"
    if x == '2':
        return "Жидкость"
    if x == '3':
        return "Молочная продукция"
    if x == '4':
        return "Мясная продукция"
    return "Овощи / фрукты"
                        
idx = np.random.randint(train.shape[0], size = 10)
for i in idx:
    print('Для названия', train['Full_descr'].iloc[i], 'класс', foo(train_res[i]))

Для названия CANDIES,NESTLE,BUTTERFINGER BAR 1 serving, 2.1 oz bar 1 serving, 1 fun size bar 0.65 oz класс Хлебобулочные изделия
Для названия BEEF,LN,TOP LN STK,BNLSS,LIP-ON,LN,1/8" FAT,ALL GRDS,,GRLLD 3 oz 1 steak класс Мясная продукция
Для названия EGG,WHITE,RAW,FRESH 1 large 1 cup класс Мясная продукция
Для названия BEANS,BLACK TURTLE,MATURE SEEDS,RAW 1 cup  класс Хлебобулочные изделия
Для названия BEEF,RND,EYE OF RND,RST,LN,1/8" FAT,SEL,RAW 1 oz 1 lb класс Молочная продукция
Для названия BEET GREENS,RAW 1 cup 1 leaf класс Молочная продукция
Для названия BEEF,SHRT LOIN,T-BONE STK,LN & FAT,0" FAT,ALL GRDS,CKD,BRLD 3 oz,  ( 1 serving ) 1 lb класс Молочная продукция
Для названия LAMB,AUS,IMP,FRSH,LEG,SIRLOIN CHOPS,BNLESS,LN,1/8"FAT,RAW 1 oz  класс Мясная продукция
Для названия KRUSTEAZ ALMOND POPPYSEED MUFFIN MIX,ARTIFICIALLY FLAV,DRY 1 serving  класс Овощи / фрукты
Для названия SILK HAZELNUT CREAMER 1 tbsp  класс Овощи / фрукты


In [12]:
class ClassifierModel():
    def __init__(self):
        self.xgb = XGBClassifier()
        #self.knn = KNeighborsClassifier(n_neighbors=8)
        #self.sgd = SGDClassifier()
        self.rf = RandomForestClassifier()
        self.log = LogisticRegression()
        
        self.log2 = LogisticRegression()
        
    def get_first_level(self, X):
        first_level = np.concatenate((
            #self.knn.predict(X).reshape(-1, 1),
            #self.sgd.predict(X).reshape(-1, 1),
            self.rf.predict(X).reshape(-1, 1),
            self.xgb.predict(X).reshape(-1, 1),
            self.log.predict(X).reshape(-1, 1)
        ), axis = 1)
        
        return first_level
    
    def fit(self, X, y):
        #self.knn.fit(X, y)
        #self.sgd.fit(X, y)
        self.rf.fit(X, y)
        self.xgb.fit(X, y)
        self.log.fit(X, y)
        
        self.log2.fit(self.get_first_level(X), y)
        
    def predict(self, X):
        first_level = self.get_first_level(X)
        result = self.log2.predict(first_level)
        
        return result
        
    def print_accuracy(self, X, y):
        y_pred = self.predict(X)
        print('Accuracy score is', accuracy_score(y_pred, y))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    train.drop(['Full_descr', 'Energ_Kcal'], axis=1).fillna(0), train_res.astype('int64'), test_size=0.33, random_state=42)

model = ClassifierModel()
model.fit(X_train, y_train)

c:\users\shepe\python\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\shepe\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\shepe\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\shepe\python\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\shepe\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lb

In [14]:
# Помотрим на показатель на обучающей выборке
model.print_accuracy(X_train, y_train)

Accuracy score is 0.7757890685142417


In [15]:
# Помотрим на показатель на отложенной выборке
model.print_accuracy(X_test, y_test)

Accuracy score is 0.509375


In [ ]:
final_model = ClassifierModel()
final_model.fit(train.drop(['Full_descr', 'Energ_Kcal'], axis=1).fillna(0), 
          train_res.astype('int64'))

final_res = final_model.predict(test.drop(['Full_descr'], axis=1).fillna(0))
pd.DataFrame(final_res).to_csv("Pred_extra_1.csv", header = ['Pred_class'], index=False)

c:\users\shepe\python\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
